In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

num_train, height, width, depth = X_train.shape
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0]

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_test)

Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
inp = Input(shape=(height, width, depth))
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size),
                      padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size),
                      padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size),
                      padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size),
                      padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)

flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out)

model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])
model.fit(X_train, Y_train,
         batch_size=batch_size, epochs=num_epochs,
         verbose=1, validation_split=0.1)

model.evaluate(X_test, Y_test, verbose=1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 764s - loss: 1.4986 - acc: 0.4521 - val_loss: 1.1190 - val_acc: 0.5998
Epoch 2/200
45000/45000 [==============================] - 752s - loss: 1.1086 - acc: 0.6070 - val_loss: 0.9140 - val_acc: 0.6820
Epoch 3/200
12480/45000 [=======>......................] - ETA: 604s - loss: 0.9726 - acc: 0.6579

In [ ]:
from keras_sequential_ascii import sequential_model_to_ascii_printout as print_model
print_model(model)

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=1)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])